In [ ]:
import numpy as np
import cv2
import time

In [ ]:
def gaussian(x, sigma):
    return (1 / (2 * np.pi * sigma**2)) * np.exp(-x**2 / (2 * sigma**2))

def bilateral_filter(image, diameter, sigma_color, sigma_space):
    height, width = image.shape
    result = np.zeros_like(image, dtype=np.float64)
    for i in range(height):
        for j in range(width):
            intensity = image[i, j]
            # Create 2D arrays representing pixel coordinates
            x_coords, y_coords = np.meshgrid(
                np.arange(max(0, i - diameter), min(height, i + diameter + 1)),
                np.arange(max(0, j - diameter), min(width, j + diameter + 1))
            )
            
            spatial_distance = np.sqrt((i - x_coords)**2 + (j - y_coords)**2)
            intensity_difference = intensity - image[x_coords, y_coords]
            spatial_weight = gaussian(spatial_distance, sigma_space)
            # spatial_weight = spatial_weight[:, :, np.newaxis]
            intensity_weight = gaussian(intensity_difference, sigma_color)

            weight = spatial_weight * intensity_weight
            weighted_sum = np.sum(image[x_coords, y_coords] * weight)
            total_weight = np.sum(weight)

            result[i, j] = weighted_sum / total_weight

    return result.astype(np.uint8)

# flash = cv2.imread("./Assignment-2/Q2/ultimate_test/2_a.jpg")
# diameter = 3
# sigma_color = 75
# sigma_space = 75
# 
# # Apply the bilateral filter
# start_time = time.time()
# filtered_image = bilateral_filter_numpy(flash, diameter, sigma_color, sigma_space)
# end_time = time.time()
# runtime = end_time - start_time
# 
# print(f"Runtime: {runtime} seconds")
# 
# # Display the original and filtered images
# cv2.imshow('Original Image', flash)
# cv2.imshow('Bilateral Filtered Image', filtered_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [ ]:
def decouple_intensity_color(image):
    # Convert the image to the LAB color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Extract the L (intensity) channel
    intensity_channel = lab_image[:, :, 0]

    # Merge the color maps with the intensity channel
    color_image = image/intensity_channel[:,:,np.newaxis]
    color_image *= 255
    return intensity_channel, color_image
# 
# # Read the image
# image = cv2.imread('./Assignment-2/Q2/ultimate_test/2_b.jpg')
# 
# # Check if the image is loaded successfully
# if image is not None:
#     # Decouple the image into intensity and color components
#     intensity, color = decouple_intensity_color(image)
# 
#     # Display the original image, intensity, and color images
#     cv2.imshow('Original Image', image)
#     cv2.imshow('Intensity', intensity)
#     cv2.imshow('Color', color)
# 
#     # Wait for a key press and close the windows
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
# else:
#     print("Error loading the image.")


In [ ]:
def decouple_largescale_detail(intensity):
    sigma_s = 105  # Spatial standard deviation
    sigma_r = 4  # Intensity standard deviation
    diameter = 0
    # large_scale_layer = cv2.bilateralFilter(intensity, d=0, sigmaColor=sigma_r, sigmaSpace=sigma_s)
    large_scale_layer = bilateral_filter(intensity, diameter=diameter, sigma_color=sigma_r, sigma_space=sigma_s)
    detail_layer = np.clip(intensity - large_scale_layer, 0, 255).astype(np.uint8)
    return large_scale_layer,detail_layer


# image = cv2.imread('./Assignment-2/Q2/ultimate_test/2_b.jpg')

# cv2.imshow("original",intensity)
# cv2.imshow("large_scale",large_scale_layer)
# cv2.imshow("detail",detail_layer)
# cv2.waitKey()
# cv2.destroyAllWindows()


In [ ]:
def combine_detail_largescale(detail_layer,large_scale_layer):
    reconstructed_image = detail_layer + large_scale_layer
    reconstructed_image = np.clip(reconstructed_image, 0, 255).astype(np.uint8)
    return reconstructed_image
# Combine the detail and large-scale layers

# Clip the reconstructed image to [0, 255] if necessary

# Display or save the reconstructed image
# cv2.imshow('Reconstructed Image', reconstructed_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# 
# If you want to save the reconstructed image
# cv2.imwrite('reconstructed_image.jpg', reconstructed_image)


In [ ]:
def combine_color_intensity(color_image,intensity_image):
    normalized_intensity = intensity_image.astype(float) / 255.0
    normalized_color_image = color_image * normalized_intensity[:, :, np.newaxis]
    normalized_color_image = np.clip(normalized_color_image, 0, 255).astype(np.uint8)
    return normalized_color_image


In [7]:
flash = cv2.imread('./ultimate_test/2_b.jpg')
no_flash = cv2.imread('./ultimate_test/2_a.jpg')
no_flash_intensity, _ = decouple_intensity_color(no_flash)
no_flash_largescale, _ = decouple_largescale_detail(no_flash_intensity)
flash_intensity,flash_color = decouple_intensity_color(flash)
_, flash_detail = decouple_largescale_detail(flash_intensity)
combined_intensity = combine_detail_largescale(flash_detail,no_flash_largescale)
filtered_image = combine_color_intensity(flash_color,combined_intensity)
cv2.imshow("no_falsh_color",no_flash_largescale)
cv2.imshow("falsh_color",flash_color)
cv2.imshow("flash_detail",flash_detail)
cv2.imshow("Flash Image",flash)
cv2.imshow("No Flash Image",no_flash)
cv2.imshow("Filtered Image",filtered_image)
cv2.waitKey(0)
cv2.destroyAllWindows()